# Gravity Model

In [8]:
import folium
import geopandas
from haversine import haversine
from matplotlib import pyplot as plt
import numpy as np
import pandas
from shapely.geometry import Point

from dpd.geometry import dms_to_dd
from dpd.wikipedia import get_wikipedia_coordinates, get_wikipedia_table
from dpd.modeling import GravityModel

pandas.set_option('display.max_columns', None)
pandas.set_option('display.max_rows', None)

In [2]:
url = "https://en.wikipedia.org/wiki/Combined_statistical_area"
csa_styled = get_wikipedia_table(url, 1, styled=True)
csa = get_wikipedia_table(url, 1)
csa.set_index("Rank", inplace=True)
csa.drop(index="143", inplace=True)
csa_styled.data.drop(index=csa_styled.data.iloc[142].name, inplace=True)
csa["2019 estimate"] = csa["2019 estimate"].map(lambda x: int(x.replace(',', '')))
coordinates = csa_styled.data["Combined\xa0statistical\xa0area"].map(
    lambda row: get_wikipedia_coordinates(row.split("href=\"")[1].split("\"")[0])
)
coordinates.index = csa.index
csa["coordinates"] = coordinates
csa = csa[csa["coordinates"] != (None, None)]
csa["dd"] = csa["coordinates"].map(lambda latlon: (dms_to_dd(latlon[0]), dms_to_dd(latlon[1])))
csa["geometry"] = csa["dd"].map(lambda x: Point(x[1], x[0]))
csa = geopandas.GeoDataFrame(csa)
csa.crs = "EPSG:4326"
csa.head()

,Combined statistical area,2019 estimate,2010 Census,Change,Constituent core-based statistical areas,coordinates,dd,geometry
Rank,,,,,,,,
1,"New York-Newark, NY-NJ-CT-PA Combined Statisti...",22589036,"22,255,491",+1.50%,"New York-Newark-Jersey City, NY-NJ-PA Metropol...","(40°48′31″N, 74°1′13.39″W)","(40.808611111111105, -74.02038611111111)",POINT (-74.02039 40.80861)
2,"Los Angeles-Long Beach, CA Combined Statistica...",18711436,"17,877,006",+4.67%,"Los Angeles-Long Beach-Anaheim, CA Metropolita...","(34°00′N, 118°12′W)","(34.0, -118.2)",POINT (-118.20000 34.00000)
3,"Chicago-Naperville, IL-IN-WI Combined Statisti...",9825325,"9,840,929",−0.16%,"Chicago-Naperville-Elgin, IL-IN-WI Metropolita...","(41°54′N, 87°39′W)","(41.9, -87.65)",POINT (-87.65000 41.90000)
4,"Washington-Baltimore-Arlington, DC-MD-VA-WV-PA...",9814928,"9,050,192",+8.45%,"Washington-Arlington-Alexandria, DC-VA-MD-WV M...","(38°58′N, 77°19′W)","(38.96666666666667, -77.31666666666666)",POINT (-77.31667 38.96667)
5,"San Jose-San Francisco-Oakland, CA Combined St...",9665887,"8,923,942",+8.31%,"San Francisco-Oakland-Berkeley, CA Metropolita...","(37°45′N, 122°17′W)","(37.75, -122.28333333333333)",POINT (-122.28333 37.75000)


In [26]:
folium_map = folium.Map(location=(39, -95), zoom_start=4)
geojson = folium.GeoJson(                                  
    csa.to_json(),
    tooltip=folium.features.GeoJsonTooltip(fields=["Combined\xa0statistical\xa0area"]),    
)                 
geojson.add_to(folium_map)
folium_map

In [4]:
# from https://pedestrianobservations.com/2020/02/13/metcalfes-law-for-high-speed-rail/
HighSpeedRailModel = GravityModel(G=75000, a=.8, b=.8, d=2)

def calculate_distance(origin, destination):
    distance = haversine(
                    origin,
                    destination
                )
    if distance == 0:
        distance = np.inf
    if distance < 500:
        distance = 500
    return distance

rows = []
for origin in csa.index:
    row = []
    for destination in csa.index:
        row.append(
            HighSpeedRailModel.compute(
                csa["2019 estimate"][origin] / 1000000,
                csa["2019 estimate"][destination] / 1000000,
                calculate_distance(
                    csa["dd"][origin],
                    csa["dd"][destination]
                )
            )
        )
    rows.append(row)

df = pandas.DataFrame(rows)
df.index = csa["Combined\xa0statistical\xa0area"]
df.columns = csa["Combined\xa0statistical\xa0area"]
df = df.astype(int)
df = df.astype(str)
df.replace("0", "", inplace=True)
df

Combined statistical area,"New York-Newark, NY-NJ-CT-PA Combined Statistical Area","Los Angeles-Long Beach, CA Combined Statistical Area","Chicago-Naperville, IL-IN-WI Combined Statistical Area","Washington-Baltimore-Arlington, DC-MD-VA-WV-PA Combined Statistical Area","San Jose-San Francisco-Oakland, CA Combined Statistical Area","Boston-Worcester-Providence, MA-RI-NH-CT Combined Statistical Area","Philadelphia-Reading-Camden, PA-NJ-DE-MD Combined Statistical Area","Miami-Port St. Lucie-Fort Lauderdale, FL Combined Statistical Area","Atlanta–Athens-Clarke County–Sandy Springs, GA-AL Combined Statistical Area","Detroit-Warren-Ann Arbor, MI Combined Statistical Area","Phoenix-Mesa, AZ Combined Statistical Area","Minneapolis-St. Paul, MN-WI Combined Statistical Area","Denver-Aurora, CO Combined Statistical Area","Cleveland-Akron-Canton, OH Combined Statistical Area","Portland-Vancouver-Salem, OR-WA Combined Statistical Area","Charlotte-Concord, NC-SC Combined Statistical Area","Salt Lake City-Provo-Orem, UT Combined Statistical Area","Columbus-Marion-Zanesville, OH Combined Statistical Area","Kansas City-Overland Park-Kansas City, MO-KS Combined Statistical Area","Las Vegas-Henderson, NV Combined Statistical Area","Cincinnati-Wilmington-Maysville, OH-KY-IN Combined Statistical Area","Raleigh-Durham-Cary, NC Combined Statistical Area","Milwaukee-Racine-Waukesha, WI Combined Statistical Area","Virginia Beach-Norfolk, VA-NC Combined Statistical Area","Greensboro–Winston-Salem–High Point, NC Combined Statistical Area","Louisville/Jefferson County–Elizabethtown–Bardstown, KY-IN Combined Statistical Area","Oklahoma City-Shawnee, OK Combined Statistical Area","Greenville-Spartanburg-Anderson, SC Combined Statistical Area","Grand Rapids-Kentwood-Muskegon, MI Combined Statistical Area","Memphis-Forrest City, TN-MS-AR Combined Statistical Area","Birmingham-Hoover-Talladega, AL Combined Statistical Area","Fresno-Madera-Hanford, CA Combined Statistical Area","Harrisburg-York-Lebanon, PA Combined Statistical Area","Buffalo-Cheektowaga-Olean, NY Combined Statistical Area","Cape Coral-Fort Myers-Naples, FL Combined Statistical Area","Albany-Schenectady, NY Combined Statistical Area","Tucson-Nogales, AZ Combined Statistical Area","Omaha-Council Bluffs-Fremont, NE-IA Combined Statistical Area","McAllen-Edinburg, TX Combined Statistical Area","Little Rock-North Little Rock, AR Combined Statistical Area","Des Moines-Ames-West Des Moines, IA Combined Statistical Area","Toledo-Findlay-Tiffin, OH Combined Statistical Area","Boise City-Mountain Home-Ontario, ID-OR Combined Statistical Area","Charleston-Huntington-Ashland, WV-OH-KY Combined Statistical Area","Lexington-Fayette–Richmond–Frankfort, KY Combined Statistical Area","Mobile-Daphne-Fairhope, AL Combined Statistical Area","Youngstown-Warren, OH-PA Combined Statistical Area","Huntsville-Decatur, AL Combined Statistical Area","Fort Wayne-Huntington-Auburn, IN Combined Statistical Area","Myrtle Beach-Conway, SC-NC Combined Statistical Area","Kalamazoo-Battle Creek-Portage, MI Combined Statistical Area","Davenport-Moline, IA-IL Combined Statistical Area","Montgomery-Selma-Alexander City, AL Combined Statistical Area","Appleton-Oshkosh-Neenah, WI Combined Statistical Area","Saginaw-Midland-Bay City, MI Combined Statistical Area","Green Bay-Shawano, WI Combined Statistical Area","Erie-Meadville, PA Combined Statistical Area","Medford-Grants Pass, OR Combined Statistical Area","Wausau-Stevens Point-Wisconsin Rapids, WI Combined Statistical Area","Springfield-Jacksonville-Lincoln, IL Combined Statistical Area","Greenville-Kinston-Washington, NC Combined Statistical Area","Fargo-Wahpeton, ND-MN Combined Statistical Area","Bloomsburg-Berwick-Sunbury, PA Combined Statistical Area","Columbia-Moberly-Mexico, MO Combined Statistical Area","Idaho Falls-Rexburg-Blackfoot, ID Combined Statistical Area","Redding-Red Bluff, CA Combined Statistical Area","State College-DuBois, PA Combined Statistical Area","Bend-P